# Proyecto II: Generación automatica de texto

En este poryecto el objetivo es desarrollar un modelo de generación aunotática de texto utilizando redes recurrentes.Para este modelo deben realizar los siguientes pasos:

1. Importar un corpus (texto con el que van a entrenar el modelo). Este corpus debe ser grande y contener mas de 100 mil palabras. Para este proyecto se pedira que entrenen con dos corpus de estilo diferente. Por ejemplo, un corpus con las obras de gabriel garcia marquez, otro corpus con poemas.
2. Preprocesar el texto. Tenga en cuenta que la entrada debe ser lo más homogenea posible, para poder entrenar las redes. Algunos modelos eliminan la puntuación, sin embargo, si esto se hace el texto generado no tendra puntuación. Yo recomiendo dejar estos elementos. Al igual que los saltos de lineas si entrenan poemas, o incluso en el texto para generar el punto aparte.
3. Preparar los datos de entrenamiento, esto inclutye los wordembeddings, o la generación de indices que puedan ser utilziados en un acapa de embedding.
4. Diseño de la arquitectura de red. Yo aconsejo utilizar un acapa de embeddings, para que el algoritmo por si solo aprenda el embedding apropiado para la tarea.
5. generación de texto. Tenga en cuenta que al entrenar se entrena en modo profesor, pero al generar se debe retroalimentar la salida del modelo como entrada para producir las palabras siguientes. Para parar su ejecución pueden generar un token de finalizacion /<end/> y solo parar cuando este token aparezca, si este es el caso ese token debe estar en el diccionario y en el texto que esten procesando. Otra forma es generar x número de palabras y parar la ejecución una vez se haya cumplido con ese número de palabras.
6. Para ejecutar el programa deben proporcionar una semilla, esta semilla debe contener palabras que esten en el vocabulario con el que se entreno, una vez hecho esto se puede correr el modelo.
7. En la entrega deben especificar paso a paso el proceso realizado, comparar los resultados del texto generado con ambos modelos entrenados (uno para cada corpus) y discutir a profundidad los resultados obtenidos, problemas y posibles mejoras.

Dado que este proyecto requiere el uso de temas no vistos a profundidad en clase, les proporciono el siguiente blog que pueden usar como base para el desarrollo del proyecto. Les recomiendo lo lean, lo sigan paso a paso, ejecuten el código que allí se encuentra y luego hagan las modificaciones necesarias para el desarrollo de su proyecto. El blog lo pueden encontrar en este [link](https://machinelearningmastery.com/how-to-develop-a-word-level-neural-language-model-in-keras/). 

El proyecto lo deben entregar el Lunes 24 de abril


# Desarrollo

Como primer paso, se definen cuatro funciones. La primera función carga y lee los archivos de texto y retorna una única lista. La segunda función, se encarga de limpiar el texto, eliminando algunos caracteres especiales y castear (casting) las palabras a minúscula, esta función retorna una lista con todas las palabras de acuerdo con el orden que fueron escritas. La tercera función se encarga de guardar el texto ya procesado en un nuevo archivo de texto. Por último, la cuarta función recibe las palabras (tokens) y una longitud definida, dicha longitud define el tamaño de la secuencia que se generara a partir de las palabras de entrada. Estas cuatro funciones componen todo el preprocesamiento de texto para cada corpus

In [1]:
def load_docs(filenames):
    complete_text = []
    for filename in filenames:
        file = open(filename, 'r')
        text = file.read()
        complete_text += text
        file.close()
    return text

def clean_text(doc):
    doc = doc.replace('--', '')
    doc = doc.replace('—', '')
    doc = doc.replace('-', '')
    doc = doc.replace('«', '')
    doc = doc.replace('»', '')
    doc = doc.replace('.', '. ')
    doc = doc.replace(' .', '.')
    doc = doc.replace('[', '')
    doc = doc.replace(']', '')
    doc = doc.replace('...', '')
    doc = doc.replace(' !', '!')
    doc = doc.replace(' ?', '?')
    doc = doc.replace(' ,', ',')
    doc = doc.replace('( ', '(')
    doc = doc.replace(' )', ')')
    tokens = doc.split()
    tokens = [token.lower() for token in tokens if token != '']
    print("Clean text completed")
    print(f'Total tokens: {len(tokens)}')
    print(f'Unique tokens: {len(set(tokens))}')
    return tokens

def save_doc(lines, filename):
    data = '\n'.join(lines)
    file = open(filename, 'w')
    file.write(data)
    file.close()

def to_sequences(length, tokens):
    sequences = []
    for i in range(length, len(tokens)):
        seq = tokens[i-length:i]
        if len(seq) == length:
            line = ' '.join(seq)
            sequences.append(line)
        else:
            sequences
    print("Sequences formed")
    print(f'Total Sequences: {len(sequences)}')
    return sequences

In [1]:
def load_doc(filename):
    # open the file as read only
    file = open(filename, 'r')
    # read all text
    text = file.read()
    # close the file
    file.close()
    return text

## Definición de una métrica 
Es importante tener en cuenta que en la generación de texto el `accuracy` no es buena medida para el rendimiento del modelo. Por ello, además de considerar el accuracy como métrica, consideraremos la perplejidad (perplexity). 

Teniendo en cuenta la entropía cruzada, la cual puede ser interpretada como el número medio de bits necesarios para almacenar la información de una variable, donde p es la distribución real de nuestro lenguaje y q es la distribución estimada de nuestro modelo. Dado que no se conoce el valor de p, dada una secuencia de palabras $\omega$ de longitud $N$ y haciendo uso del teorema de Shannon-McMillan-Breiman, se puede aproximar la entropía cruzada por palabra como: 

$$H(p,q) \approx  -\dfrac{1}{N}*log_{2}q(\omega)$$ 

Teniendo esto en cuenta, podemos definir la perplejidad como el exponencial de la entropía cruzada 

$$P(p,q) = e^{H(p,q)}$$

Lo cual no indica el numero promedio de palabras que se pueden codificar usando $H(\omega)$ bits, es decir, representa el número de palabras que son posibles en cada punto de la secuencia. Para dejar en claro esto, si se tiene una perplejidad de 50, quiere decir que cada vez que el modelo intenta adivinar la siguiente palabra, esta tan confundido como si tuviera que elegir entre 50 palabras. Podemos ver que, entre menor sea el número de perplejidad, mejor será el rendimiento del modelo. No obstante, una perplejidad de 1 nos indicaría un sobreajuste en los datos, sin embargo, este es un caso aislado. 

Ahora, definimos la siguiente función para calcular la perplejidad durante el entrenamiento de cada uno de los modelos. 

In [2]:
import tensorflow as tf


def perplexity(y_true, y_pred):
    cross_entropy = tf.losses.categorical_crossentropy(y_true, y_pred)
    perplexity = tf.exp(tf.reduce_mean(cross_entropy))
    return perplexity

## Preprocesamiento de datos 

Se consideraron dos textos. El primero de ellos está compuesto por la saga de Harry Potter. El segundo es el libro El Código Da Vinci. Ambos textos cargados y se aplica el preprocesamiento a cada uno. Posteriormente, el texto limpio es guardado en un nuevo archivo de texto. 

### Corpus 1 

Podemos observar que este corpus cuenta con 208.782 palabras, un vocabulario (Palabras sin repetir) de 26.661 y en total se forman 208.731 secuencias de 51 palabras. 

In [3]:
in_filenames = ['Books_data/HarryPotter/J.K. Rowling - Harry Potter 1 - La Piedra Filosofal.txt',
               'Books_data/HarryPotter/J.K. Rowling - Harry Potter 2 - La Cámara Secreta.txt',
               'Books_data/HarryPotter/J.K. Rowling - Harry Potter 3 - El Prisionero de Azkaban.txt',
               'Books_data/HarryPotter/J.K. Rowling - Harry Potter 4 - El Cáliz de Fuego.txt',
               'Books_data/HarryPotter/J.K. Rowling - Harry Potter 5 - La Orden del Fenix.txt',
               'Books_data/HarryPotter/J.K. Rowling - Harry Potter 6 - El Misterio del Príncipe.txt',
               'Books_data/HarryPotter/J.K. Rowling - Harry Potter 7 - Las Reliquias de la Muerte.txt']
doc = load_docs(in_filenames)
tokens = clean_text(doc)
lines = to_sequences(51, tokens)

out_filename = 'HarryPoter_CleanText.txt'
save_doc(lines, out_filename)

Clean text completed
Total tokens: 208782
Unique tokens: 26661
Sequences formed
Total Sequences: 208731


### Corpus 2 

Podemos observar que este corpus cuenta con 153.890 palabras, un vocabulario (Palabras sin repetir) de 22.182 y en total se forman 153.839 secuencias de 51 palabras. 

In [5]:
in_filename = 'Books_data/El Codigo Da Vinci.txt'
doc = load_doc(in_filename)
tokens = clean_text(doc)
lines = to_sequences(51, tokens)

out_filename = 'CodigoDaVinci_CleanText.txt'
save_doc(lines, out_filename)

Clean text completed
Total tokens: 153890
Unique tokens: 22182
Sequences formed
Total Sequences: 153839


## Modelos para el corpus 1 (Harry Potter) 
Primero, se configura el uso de GPU para acelerar el tiempo de ejecución (entrenamiento) del modelo.

In [3]:
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)

Se carga el archivo que contiene el texto ya procesado y se realiza la tokenizacion del mismo. También, se definen las secuencias de acuerdo con los tokens definidos (índices). 

In [4]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Embedding

from numpy import array
from pickle import dump

in_filename = 'HarryPoter_CleanText.txt'
doc = load_doc(in_filename)
lines = doc.split('\n')

tokenizer = Tokenizer()
tokenizer.fit_on_texts(lines)
sequences = tokenizer.texts_to_sequences(lines)
sequences = [sequence for sequence in sequences if len(sequence) == 51]
vocab_size = len(tokenizer.word_index) + 1

### Primer modelo

Se define el modelo el cual consta de una capa de embedding, dos capas de Long short-term memory (LSTM) y 2 capas densas, la primera con una función de activación relu y la segunda con una función de activación softmax. Como optimizador se utiliza Adam.

In [5]:
# define model
seq_length = 50 #X.shape[1]

model = Sequential()
model.add(Embedding(vocab_size, 50, input_length=seq_length))
model.add(LSTM(80, return_sequences=True))
model.add(LSTM(80))
model.add(Dense(50, activation='relu'))
model.add(Dense(vocab_size, activation='softmax'))
print(model.summary())
# compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy', perplexity,])

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 50, 50)            886050    
                                                                 
 lstm (LSTM)                 (None, 50, 80)            41920     
                                                                 
 lstm_1 (LSTM)               (None, 80)                51520     
                                                                 
 dense (Dense)               (None, 50)                4050      
                                                                 
 dense_1 (Dense)             (None, 17721)             903771    
                                                                 
Total params: 1,887,311
Trainable params: 1,887,311
Non-trainable params: 0
_________________________________________________________________
None


Dado el tamaño de los datos, la arquitectura del modelo y la capacidad computacional disponible, los datos son divididos en 4 secciones y cada sección se presenta al modelo con un batch size de 128 y 40 épocas. Los resultados del entrenamiento son almacenados en los archivos `model_HarryPotter.h5` y `tokenizer_HarryPotter.pkl`. 

In [ ]:
size_data = len(sequences)//4

for i in range(size_data,len(sequences)+1, size_data):
    sub_sequences = sequences[i-size_data:i]
    sequences_aux = array(sub_sequences)
    
    X, y = sequences_aux[:,:-1], sequences_aux[:,-1]
    y = to_categorical(y, num_classes=vocab_size)

    model.fit(X, y, batch_size=128, epochs=40)
    
    # save the model to file
    model.save('model_HarryPotter.h5')
    # save the tokenizer
    dump(tokenizer, open('tokenizer_HarryPotter.pkl', 'wb'))

2022-04-29 20:16:22.853824: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 3697096788 exceeds 10% of free system memory.
2022-04-29 20:16:27.278876: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 3697096788 exceeds 10% of free system memory.


Epoch 1/40
408/408 [==============================] - 10s 15ms/step - loss: 7.3191 - accuracy: 0.0438 - perplexity: 2186.3201
Epoch 2/40
408/408 [==============================] - 6s 15ms/step - loss: 6.7323 - accuracy: 0.0474 - perplexity: 864.6395
Epoch 3/40
408/408 [==============================] - 6s 15ms/step - loss: 6.5118 - accuracy: 0.0636 - perplexity: 692.0182
Epoch 4/40
408/408 [==============================] - 6s 15ms/step - loss: 6.3132 - accuracy: 0.0694 - perplexity: 567.4196
Epoch 5/40
408/408 [==============================] - 6s 16ms/step - loss: 6.1697 - accuracy: 0.0727 - perplexity: 489.3873
Epoch 6/40
408/408 [==============================] - 6s 16ms/step - loss: 6.0364 - accuracy: 0.0797 - perplexity: 431.1900
Epoch 7/40
408/408 [==============================] - 6s 15ms/step - loss: 5.2971 - accuracy: 0.1308 - perplexity: 204.5429
Epoch 14/40
408/408 [==============================] - 6s 15ms/step - loss: 5.2049 - accuracy: 0.1361 - perplexity: 186.4925
Epoch

In [18]:
size_data = int(len(sequences)*(3/4))


sub_sequences = sequences[size_data:]
sequences_aux = array(sub_sequences)
    
X, y = sequences_aux[:,:-1], sequences_aux[:,-1]
y = to_categorical(y, num_classes=vocab_size)


model = load_model('model_HarryPotter.h5', custom_objects={'perplexity':perplexity}, compile=True)
tokenizer = load(open('tokenizer_HarryPotter.pkl', 'rb'))


model.fit(X, y, batch_size=128, epochs=40)
    
# save the model to file
model.save('model_HarryPotter.h5')
# save the tokenizer
dump(tokenizer, open('tokenizer_HarryPotter.pkl', 'wb'))

2022-04-30 04:55:47.100893: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 3697167672 exceeds 10% of free system memory.
2022-04-30 04:55:51.406761: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 3697167672 exceeds 10% of free system memory.


Epoch 1/40
408/408 [==============================] - 8s 16ms/step - loss: 4.0449 - accuracy: 0.2918 - perplexity: 59.1985
Epoch 2/40
408/408 [==============================] - 6s 16ms/step - loss: 4.0104 - accuracy: 0.2939 - perplexity: 57.2866
Epoch 3/40
408/408 [==============================] - 6s 16ms/step - loss: 3.9729 - accuracy: 0.3002 - perplexity: 55.1830
Epoch 4/40
408/408 [==============================] - 6s 16ms/step - loss: 3.9408 - accuracy: 0.3020 - perplexity: 53.3407
Epoch 5/40
408/408 [==============================] - 6s 16ms/step - loss: 3.9027 - accuracy: 0.3057 - perplexity: 51.2433
Epoch 6/40
408/408 [==============================] - 6s 16ms/step - loss: 3.8689 - accuracy: 0.3110 - perplexity: 49.4850
Epoch 7/40
408/408 [==============================] - 6s 16ms/step - loss: 3.8376 - accuracy: 0.3133 - perplexity: 48.1481
Epoch 8/40
408/408 [==============================] - 6s 16ms/step - loss: 3.8090 - accuracy: 0.3154 - perplexity: 46.9789
Epoch 9/40
408/4

### Segundo modelo
Al igual que el primer modelo, esta arquitectura consta de una capa de embedding, dos capas de LSTM y 2 capas densas con funciones de activación relu y softmax respectivamente. Para este modelo se consideró una menor cantidad de neuronas, disminuyendo los parámetros a entrenar en un 23% aproximadamente 

In [5]:
# define model
seq_length = 50 #X.shape[1]

model3 = Sequential()
model3.add(Embedding(vocab_size, 50, input_length=seq_length))
model3.add(LSTM(20, return_sequences=True))
model3.add(LSTM(20))
model3.add(Dense(30, activation='relu'))
model3.add(Dense(vocab_size, activation='softmax'))
print(model3.summary())
# compile model
model3.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy', perplexity,])

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 50, 50)            886050    
                                                                 
 lstm (LSTM)                 (None, 50, 20)            5680      
                                                                 
 lstm_1 (LSTM)               (None, 20)                3280      
                                                                 
 dense (Dense)               (None, 30)                630       
                                                                 
 dense_1 (Dense)             (None, 17721)             549351    
                                                                 
Total params: 1,444,991
Trainable params: 1,444,991
Non-trainable params: 0
_________________________________________________________________
None


Al igual que en el modelo anterior, dadas las capacidades computaciones, los datos son divididos y presentados al modelo en 4 secciones con un batch size de 128 y 40 épocas. Los resultados del modelo son almacenados en los archivos `model2_HarryPotter.h5` y `tokenizer2_HarryPotter.pkl`.

In [ ]:
size_data = len(sequences)//4

for i in range(size_data,len(sequences)+1, size_data):
    sub_sequences = sequences[i-size_data:i]
    sequences_aux = array(sub_sequences)
    
    X, y = sequences_aux[:,:-1], sequences_aux[:,-1]
    y = to_categorical(y, num_classes=vocab_size)

    model3.fit(X, y, batch_size=128, epochs=40)
    
    # save the model to file
    model3.save('model2_HarryPotter.h5')
    # save the tokenizer
    dump(tokenizer, open('tokenizer2_HarryPotter.pkl', 'wb'))

2022-04-29 21:18:53.879613: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 3697096788 exceeds 10% of free system memory.
2022-04-29 21:18:58.306205: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 3697096788 exceeds 10% of free system memory.


Epoch 1/40
408/408 [==============================] - 9s 13ms/step - loss: 7.4211 - accuracy: 0.0449 - perplexity: 2863.1711
Epoch 2/40
408/408 [==============================] - 5s 13ms/step - loss: 6.7393 - accuracy: 0.0463 - perplexity: 866.3502
Epoch 3/40
408/408 [==============================] - 5s 13ms/step - loss: 6.6363 - accuracy: 0.0521 - perplexity: 785.2633
Epoch 4/40
408/408 [==============================] - 5s 13ms/step - loss: 6.5011 - accuracy: 0.0634 - perplexity: 684.5125
Epoch 5/40
408/408 [==============================] - 5s 13ms/step - loss: 6.3538 - accuracy: 0.0661 - perplexity: 589.8724
Epoch 6/40
408/408 [==============================] - 5s 13ms/step - loss: 6.2188 - accuracy: 0.0693 - perplexity: 515.4156
Epoch 7/40
408/408 [==============================] - 5s 13ms/step - loss: 6.0890 - accuracy: 0.0771 - perplexity: 453.2469
Epoch 8/40
408/408 [==============================] - 5s 13ms/step - loss: 5.9616 - accuracy: 0.0860 - perplexity: 400.4470
Epoch 9

2022-04-29 21:22:41.868880: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 3697096788 exceeds 10% of free system memory.
2022-04-29 21:22:46.287030: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 3697096788 exceeds 10% of free system memory.


Epoch 1/40
408/408 [==============================] - 5s 13ms/step - loss: 7.2949 - accuracy: 0.0656 - perplexity: 14176.2422
Epoch 2/40
408/408 [==============================] - 5s 13ms/step - loss: 6.3057 - accuracy: 0.0820 - perplexity: 571.3538
Epoch 3/40
408/408 [==============================] - 5s 13ms/step - loss: 6.1146 - accuracy: 0.0933 - perplexity: 468.9853
Epoch 4/40
408/408 [==============================] - 5s 13ms/step - loss: 5.9541 - accuracy: 0.1053 - perplexity: 398.6401
Epoch 5/40
408/408 [==============================] - 5s 13ms/step - loss: 5.8061 - accuracy: 0.1153 - perplexity: 342.9427
Epoch 6/40
408/408 [==============================] - 5s 13ms/step - loss: 5.6672 - accuracy: 0.1245 - perplexity: 297.6137
Epoch 7/40
408/408 [==============================] - 5s 13ms/step - loss: 5.5416 - accuracy: 0.1330 - perplexity: 262.2934
Epoch 8/40
408/408 [==============================] - 5s 13ms/step - loss: 5.4206 - accuracy: 0.1402 - perplexity: 232.6329
Epoch 

2022-04-29 21:26:26.760787: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 3697096788 exceeds 10% of free system memory.


Epoch 1/40
408/408 [==============================] - 5s 13ms/step - loss: 7.5595 - accuracy: 0.0612 - perplexity: 79606.2031
Epoch 2/40
408/408 [==============================] - 5s 13ms/step - loss: 6.5005 - accuracy: 0.0838 - perplexity: 691.2253
Epoch 3/40
408/408 [==============================] - 5s 13ms/step - loss: 6.2476 - accuracy: 0.0917 - perplexity: 532.5765
Epoch 4/40
408/408 [==============================] - 5s 13ms/step - loss: 6.0664 - accuracy: 0.1002 - perplexity: 446.4307
Epoch 5/40
408/408 [==============================] - 5s 13ms/step - loss: 5.9113 - accuracy: 0.1099 - perplexity: 383.6451
Epoch 6/40
408/408 [==============================] - 5s 13ms/step - loss: 5.7713 - accuracy: 0.1191 - perplexity: 331.6772
Epoch 7/40
408/408 [==============================] - 5s 13ms/step - loss: 5.6400 - accuracy: 0.1280 - perplexity: 292.1404
Epoch 8/40
408/408 [==============================] - 5s 13ms/step - loss: 5.5151 - accuracy: 0.1357 - perplexity: 256.9824
Epoch 

In [7]:
size_data = int(len(sequences)*(3/4))


sub_sequences = sequences[size_data:]
sequences_aux = array(sub_sequences)
    
X, y = sequences_aux[:,:-1], sequences_aux[:,-1]
y = to_categorical(y, num_classes=vocab_size)


model = load_model('model2_HarryPotter.h5', custom_objects={'perplexity':perplexity}, compile=True)
tokenizer = load(open('tokenizer2_HarryPotter.pkl', 'rb'))


model.fit(X, y, batch_size=128, epochs=40)
    
# save the model to file
model.save('model2_HarryPotter.h5')
# save the tokenizer
dump(tokenizer, open('tokenizer2_HarryPotter.pkl', 'wb'))

2022-04-30 04:50:04.745587: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 3697167672 exceeds 10% of free system memory.
2022-04-30 04:50:09.150621: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 3697167672 exceeds 10% of free system memory.


Epoch 1/40
408/408 [==============================] - 9s 14ms/step - loss: 7.6512 - accuracy: 0.0602 - perplexity: 9830.4658
Epoch 2/40
408/408 [==============================] - 6s 14ms/step - loss: 6.7730 - accuracy: 0.0919 - perplexity: 931.9248
Epoch 3/40
408/408 [==============================] - 6s 14ms/step - loss: 6.4982 - accuracy: 0.1040 - perplexity: 698.9741
Epoch 4/40
408/408 [==============================] - 6s 14ms/step - loss: 6.3044 - accuracy: 0.1118 - perplexity: 578.8286
Epoch 5/40
408/408 [==============================] - 6s 14ms/step - loss: 6.1460 - accuracy: 0.1194 - perplexity: 489.7347
Epoch 6/40
408/408 [==============================] - 6s 14ms/step - loss: 6.0046 - accuracy: 0.1245 - perplexity: 425.1531
Epoch 7/40
408/408 [==============================] - 6s 14ms/step - loss: 5.8736 - accuracy: 0.1324 - perplexity: 372.6039
Epoch 8/40
408/408 [==============================] - 6s 14ms/step - loss: 5.7545 - accuracy: 0.1402 - perplexity: 331.1023
Epoch 9

## Modelos para el corpus 2 (Código Da Vinci) 
Configuramos el uso de la GPU. 

In [3]:
#import tensorflow as tf
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)

Cargamos el archivo que contiene los datos limpios, se realiza la tokenizacion y se crean las secuencias. 

In [4]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Embedding

from numpy import array
from pickle import dump


in_filename = 'CodigoDaVinci_CleanText.txt'
doc2 = load_doc(in_filename)
lines2 = doc2.split('\n')

tokenizer2 = Tokenizer()
tokenizer2.fit_on_texts(lines2)
sequences2 = tokenizer2.texts_to_sequences(lines2)
sequences2 = [sequence for sequence in sequences2 if len(sequence) == 51]
vocab_size2 = len(tokenizer2.word_index) + 1

### Primer modelo 
Se define el modelo. Observe que al primr modelo del copus 1.

In [5]:
# define model
seq_length2 = 50 #X.shape[1]

model2 = Sequential()
model2.add(Embedding(vocab_size2, 50, input_length=seq_length2))
model2.add(LSTM(80, return_sequences=True))
model2.add(LSTM(80))
model2.add(Dense(50, activation='relu'))
model2.add(Dense(vocab_size2, activation='softmax'))
print(model2.summary())
# compile model
model2.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy', perplexity,])

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 50, 50)            774550    
                                                                 
 lstm (LSTM)                 (None, 50, 80)            41920     
                                                                 
 lstm_1 (LSTM)               (None, 80)                51520     
                                                                 
 dense (Dense)               (None, 50)                4050      
                                                                 
 dense_1 (Dense)             (None, 15491)             790041    
                                                                 
Total params: 1,662,081
Trainable params: 1,662,081
Non-trainable params: 0
_________________________________________________________________
None


Para el caso de este corpus, dado que es más pequeño que el primero (cuenta con tres cuartas partes de la cantidad de palabras), los datos se dividieron en 3 secciones y, de igual manera, se entrenó con un batch size de 128 con la diferencia de que el entrenamiento se realiza por 50 épocas. Los resultados del modelo se almacenan en los archivos `model_DaVinci.h5` y `tokenizer_DaVinci.pkl` 

In [6]:
size_data = len(sequences2)//3

for i in range(size_data,len(sequences2)+1, size_data):
    sub_sequences2 = sequences2[i-size_data:i]
    sequences_aux2 = array(sub_sequences2)
    
    X, y = sequences_aux2[:,:-1], sequences_aux2[:,-1]
    y = to_categorical(y, num_classes=vocab_size2)

    model2.fit(X, y, batch_size=128, epochs=50)
    
# save the model to file
model2.save('model_DaVinci.h5')
# save the tokenizer
dump(tokenizer2, open('tokenizer_DaVinci.pkl', 'wb'))

2022-04-29 01:22:46.152647: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 2710367324 exceeds 10% of free system memory.
2022-04-29 01:22:49.461103: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 2710367324 exceeds 10% of free system memory.


Epoch 1/50
342/342 [==============================] - 9s 15ms/step - loss: 7.2695 - accuracy: 0.0527 - perplexity: 2087.9705
Epoch 2/50
342/342 [==============================] - 5s 15ms/step - loss: 6.6903 - accuracy: 0.0555 - perplexity: 831.7921
Epoch 3/50
342/342 [==============================] - 5s 15ms/step - loss: 6.4557 - accuracy: 0.0742 - perplexity: 652.9443
Epoch 4/50
342/342 [==============================] - 5s 15ms/step - loss: 6.2426 - accuracy: 0.0795 - perplexity: 529.7634
Epoch 5/50
342/342 [==============================] - 5s 15ms/step - loss: 6.1127 - accuracy: 0.0836 - perplexity: 464.4283
Epoch 6/50
342/342 [==============================] - 5s 15ms/step - loss: 5.9833 - accuracy: 0.0908 - perplexity: 407.9745
Epoch 7/50
342/342 [==============================] - 5s 15ms/step - loss: 5.8426 - accuracy: 0.1020 - perplexity: 355.9132
Epoch 8/50
342/342 [==============================] - 5s 15ms/step - loss: 5.7216 - accuracy: 0.1112 - perplexity: 313.1197
Epoch 9

2022-04-29 01:27:05.510888: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 2710367324 exceeds 10% of free system memory.
2022-04-29 01:27:08.796385: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 2710367324 exceeds 10% of free system memory.


Epoch 1/50
342/342 [==============================] - 5s 15ms/step - loss: 7.7337 - accuracy: 0.0557 - perplexity: 59893.7969
Epoch 2/50
342/342 [==============================] - 5s 15ms/step - loss: 6.5630 - accuracy: 0.0827 - perplexity: 735.3923
Epoch 3/50
342/342 [==============================] - 5s 15ms/step - loss: 6.3054 - accuracy: 0.0905 - perplexity: 564.2772
Epoch 4/50
342/342 [==============================] - 5s 15ms/step - loss: 6.1314 - accuracy: 0.0970 - perplexity: 474.3522
Epoch 5/50
342/342 [==============================] - 5s 15ms/step - loss: 5.9900 - accuracy: 0.1039 - perplexity: 413.1138
Epoch 6/50
342/342 [==============================] - 5s 15ms/step - loss: 5.8592 - accuracy: 0.1108 - perplexity: 359.6851
Epoch 7/50
342/342 [==============================] - 5s 15ms/step - loss: 5.7423 - accuracy: 0.1192 - perplexity: 320.8093
Epoch 8/50
342/342 [==============================] - 5s 15ms/step - loss: 5.6312 - accuracy: 0.1287 - perplexity: 289.1989
Epoch 

2022-04-29 01:31:21.126578: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 2710367324 exceeds 10% of free system memory.


Epoch 1/50
342/342 [==============================] - 5s 15ms/step - loss: 7.1849 - accuracy: 0.0542 - perplexity: 70771.8516
Epoch 2/50
342/342 [==============================] - 5s 15ms/step - loss: 6.4129 - accuracy: 0.0708 - perplexity: 632.7710
Epoch 3/50
342/342 [==============================] - 5s 15ms/step - loss: 6.2338 - accuracy: 0.0809 - perplexity: 531.6660
Epoch 4/50
342/342 [==============================] - 5s 15ms/step - loss: 6.1066 - accuracy: 0.0866 - perplexity: 466.0289
Epoch 5/50
342/342 [==============================] - 5s 15ms/step - loss: 5.9986 - accuracy: 0.0908 - perplexity: 415.7966
Epoch 6/50
342/342 [==============================] - 5s 15ms/step - loss: 5.8901 - accuracy: 0.0977 - perplexity: 370.9175
Epoch 7/50
342/342 [==============================] - 5s 15ms/step - loss: 5.7902 - accuracy: 0.1048 - perplexity: 337.8956
Epoch 8/50
342/342 [==============================] - 5s 15ms/step - loss: 5.6966 - accuracy: 0.1109 - perplexity: 307.5073
Epoch 

### Segundo modelo
Para este modelo, se tomó como base la arquitectura anterior, se añadió una capa LSTM con 50 neuronas y se disminuyó el número de neuronas en cada una de las capas.  

In [7]:
# define model
seq_length2 = 50 #X.shape[1]

model4 = Sequential()
model4.add(Embedding(vocab_size2, 50, input_length=seq_length2))
model4.add(LSTM(80, return_sequences=True))
model4.add(LSTM(65, return_sequences=True))
model4.add(LSTM(50))
model4.add(Dense(50, activation='relu'))
model4.add(Dense(vocab_size2, activation='softmax'))
print(model4.summary())
# compile model
model4.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy', perplexity,])

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 50, 50)            774550    
                                                                 
 lstm_2 (LSTM)               (None, 50, 80)            41920     
                                                                 
 lstm_3 (LSTM)               (None, 50, 65)            37960     
                                                                 
 lstm_4 (LSTM)               (None, 50)                23200     
                                                                 
 dense_2 (Dense)             (None, 50)                2550      
                                                                 
 dense_3 (Dense)             (None, 15491)             790041    
                                                                 
Total params: 1,670,221
Trainable params: 1,670,221
No

Los datos son presentados al modelo en 3 secciones y se entrena con un batch size de 128 por 50 épocas. Los resultados del modelo son almacenados en los archivos `model2_DaVinci.h5` y `tokenizer2_DaVinci.pkl`.

In [8]:
size_data = len(sequences2)//3

for i in range(size_data,len(sequences2)+1, size_data):
    sub_sequences2 = sequences2[i-size_data:i]
    sequences_aux2 = array(sub_sequences2)
    
    X, y = sequences_aux2[:,:-1], sequences_aux2[:,-1]
    y = to_categorical(y, num_classes=vocab_size2)

    model4.fit(X, y, batch_size=128, epochs=50)
    
# save the model to file
model4.save('model2_DaVinci.h5')
# save the tokenizer
dump(tokenizer2, open('tokenizer2_DaVinci.pkl', 'wb'))

Epoch 1/50
342/342 [==============================] - 10s 18ms/step - loss: 7.2871 - accuracy: 0.0537 - perplexity: 2203.3335
Epoch 2/50
342/342 [==============================] - 6s 17ms/step - loss: 6.7697 - accuracy: 0.0541 - perplexity: 897.4904
Epoch 3/50
342/342 [==============================] - 6s 17ms/step - loss: 6.7442 - accuracy: 0.0539 - perplexity: 872.8828
Epoch 4/50
342/342 [==============================] - 6s 17ms/step - loss: 6.7349 - accuracy: 0.0541 - perplexity: 862.5467
Epoch 5/50
342/342 [==============================] - 6s 17ms/step - loss: 6.7037 - accuracy: 0.0540 - perplexity: 837.5724
Epoch 6/50
342/342 [==============================] - 6s 17ms/step - loss: 6.6098 - accuracy: 0.0579 - perplexity: 764.6805
Epoch 7/50
342/342 [==============================] - 6s 17ms/step - loss: 6.4026 - accuracy: 0.0752 - perplexity: 620.6011
Epoch 8/50
342/342 [==============================] - 6s 17ms/step - loss: 6.2160 - accuracy: 0.0808 - perplexity: 515.8389
Epoch 

# Generación de texto
Importamos las librerías necesarias para la generación de texto

In [8]:
from random import randint
from pickle import load

from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences

from numpy import argmax

In [9]:
def load_doc(filename):
    # open the file as read only
    file = open(filename, 'r')
    # read all text
    text = file.read()
    # close the file
    file.close()
    return text


def generate_seq(model, tokenizer, seq_length, seed_text, n_words):
    result = list()
    in_text = seed_text
    # generate a fixed number of words
    for _ in range(n_words):
        # encode the text as integer
        encoded = tokenizer.texts_to_sequences([in_text])[0]
        # truncate sequences to a fixed length
        encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre')
        # predict probabilities for each word
        predict_yhat = model.predict(encoded, verbose=0)
        yhat = argmax(predict_yhat, axis=1)
        # map predicted word index to word
        out_word = ''
        for word, index in tokenizer.word_index.items():
            if index == yhat:
                out_word = word
                break
        # append to input
        in_text += ' ' + out_word
        result.append(out_word)
    return ' '.join(result)

## Corpus 1 - Modelo 1

In [10]:
# load cleaned text sequences
in_filename = 'HarryPoter_CleanText.txt'
doc = load_doc(in_filename)
lines = doc.split('\n')
seq_length = len(lines[0].split()) - 1

# load the model
model = load_model('model_HarryPotter.h5', custom_objects={'perplexity':perplexity})

# load the tokenizer
tokenizer = load(open('tokenizer_HarryPotter.pkl', 'rb'))

In [11]:
# select a seed text
seed_text = lines[randint(0,len(lines))]
print(seed_text + '\n')

# generate new text
generated = generate_seq(model, tokenizer, seq_length, seed_text, 50)
print(generated)

para que su luz llegara hasta el fondo. vio un destello rojo intenso, una espada con relumbrantes rubíes en la empuñadura. la espada de gryffindor yacía en el fondo del agua. casi sin respirar, el muchacho se quedó mirándola fijamente. ¿cómo era posible? ¿cómo podía haber acabado en el fondo de

los jardines por la diadema se había llegado y se quedó mcgonagall se había llegado y se había llegado y se quedó mcgonagall se había llegado y se quedó mcgonagall se había llegado y se había llegado y se había llegado y se quedó mcgonagall se había llegado y se


## Corpus 1 - Modelo 2

In [25]:
# load cleaned text sequences
in_filename = 'HarryPoter_CleanText.txt'
doc = load_doc(in_filename)
lines2 = doc.split('\n')
seq_length2 = len(lines[0].split()) - 1

# load the model
model2 = load_model('model2_HarryPotter.h5', custom_objects={'perplexity':perplexity})

# load the tokenizer
tokenizer2 = load(open('tokenizer2_HarryPotter.pkl', 'rb'))

In [26]:
# select a seed text
seed_text2 = lines2[randint(0,len(lines2))]
print(seed_text2 + '\n')

# generate new text
generated = generate_seq(model2, tokenizer2, seq_length2, seed_text2, 50)
print(generated)

parte porque olía a gato: era la misma en que habían dormido la noche de la copa del mundo de quidditch. ¿el dueño de esta tienda no era un tal perkins del ministerio? preguntó mientras liberaba las piquetas. sí, pero por lo visto ya no la quería, porque tiene lumbago explicó

a harry y la profesora mcgonagall se había quedado para morir y no había muerto se había quedado en el castillo y no había muerto se había quedado bien sino que se había quedado prohibido de slytherin lo serpiente que se había quedado y a su lado y harry se


## Corpus 2 - Modelo 1

In [21]:
# load cleaned text sequences
in_filename = 'CodigoDaVinci_CleanText.txt'
doc3 = load_doc(in_filename)
lines3 = doc3.split('\n')
seq_length3 = len(lines3[0].split()) - 1

# load the model
model3 = load_model('model_DaVinci.h5', custom_objects={'perplexity':perplexity})

# load the tokenizer
tokenizer3 = load(open('tokenizer_DaVinci.pkl', 'rb'))

In [22]:
# select a seed text
seed_text3 = lines3[randint(0,len(lines3))]
print(seed_text3 + '\n')

# generate new text
generated3 = generate_seq(model3, tokenizer3, seq_length3, seed_text3, 50)
print(generated3)

para evitar que los demás le robaran las ideas. el caso era que el artista hacía siempre lo que le venía en gana. * * * en su fuero interno, sophie se alegró al ver que robert había captado lo que había querido decir. las primeras palabras puedo leerlas más o

charles de orbes principia mahematica a pesar de allí langdon se había abierto en la tumba de la abadía de westminster gettum se había convertido en consecuencia en dirección de la espalda de kent británico cuerpos de pie a la iglesia del temple y el poema está en el chasquido


Para el primer corpus, podemos ver que, el primer modelo termino con una perplejidad de 19, mientras que el segundo obtuvo un valor de 42.  

Analizando el texto generado con cada uno de los modelos, podemos ver que, para este caso, aunque se tiene más coherencia, el valor de perplejidad hace que sea más probable la repetición de una secuencia. En el segundo modelo, la secuencia, dado su alto valor de perplejidad presenta menos coherencia  

## Corpus 2 - Modelo 2

In [16]:
# load cleaned text sequences
in_filename = 'CodigoDaVinci_CleanText.txt'
doc4 = load_doc(in_filename)
lines4 = doc4.split('\n')
seq_length4 = len(lines4[0].split()) - 1

# load the model
model4 = load_model('model2_DaVinci.h5', custom_objects={'perplexity':perplexity})

# load the tokenizer
tokenizer4 = load(open('tokenizer2_DaVinci.pkl', 'rb'))

In [17]:
# select a seed text
seed_text4 = lines4[randint(0,len(lines4))]
print(seed_text4 + '\n')

# generate new text
generated4 = generate_seq(model4, tokenizer4, seq_length4, seed_text4, 50)
print(generated4)

su identidad. ¿está seguro de que quiere que sea silas quien haga el trabajo? le había preguntado hacía menos de media hora, cuando le había ordenado que robaran la clave. puedo hacerlo yo mismo. pero el maestro había sido muy claro. silas nos ha servido sin problemas con los cuatro miembros

del priorato de sir leigh que encontramos los apetece a los demás espada en la emoción que dudó sabía encontrar la iglesia que había dicho que incluía favor que sus muros de voz tanteó sophie de capaces siguiendo su familia se puede decir el orbe que es un reverendo s


Podemos observar, luego del entrenamiento de cada modelo para este corpus que, el primer modelo termino con una perplejidad de 24 aproximadamente, mientras que el segundo con un valor de 12. Razón por la cual, se espera que el segundo modelo tenga un mejor resultado en la generación de texto. 

Podemos ver que, a pesar de la falta de conexidad entre la secuencia, el segundo modelo otorga un mejor resultado al momento de evaluar el tema sobre el cual se habla 

Camilo Andres Gomez Vargas
